# Colab Preperations

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd 'gdrive/My Drive/Colab Notebooks/MSC-RecSys-Project'
!pip install optuna
!pip install pytorch-ignite

# Imports

In [5]:
import torch
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from src.hyperparams_tuning import tune_params, final_train

In [6]:
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


# MovieLens 1M

In [7]:
max_epochs_ml = 1

## HyperParameter Tuning

In [8]:
n_trials = 1

study_ml, df_tuning_results_ml = tune_params(
    model_name='MF', 
    dataset_name='movielens', 
    max_epochs=max_epochs_ml,
    n_trials=n_trials, 
    device=device
    )

[I 2022-02-08 21:27:24,307] A new study created in memory with name: no-name-3fff90ad-4d8d-4068-b76d-2c99c083ef3e


Movie-lens file already exists
Movielens ratings shape: (1000209, 4)
Number of users: 6040
Number of items: 3706
Training Results - Epoch[1] Avg loss: 4.77
Validation Results - Epoch[1] Avg loss: 4.72


[I 2022-02-08 21:28:07,862] Trial 0 finished with value: 4.723997829209586 and parameters: {'learning_rate': 0.1, 'optimizer': 'SGD', 'latent_dim': 10, 'batch_size': 512}. Best is trial 0 with value: 4.723997829209586.


Validation Results - Epoch[1] MRR: 0.75


In [9]:
best_params_ml = study_ml.best_params
df_tuning_results_ml.sort_values(by='value').head(15)

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_latent_dim,params_learning_rate,params_optimizer,state
0,0,4.723998,2022-02-08 21:27:24.308328,2022-02-08 21:28:07.859709,0 days 00:00:43.551381,512,10,0.1,SGD,COMPLETE


## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [10]:
test_loss_ml, final_model_ml, test_mrr_ml = final_train(
    model_name='MF', 
    dataset_name='movielens', 
    best_params=best_params_ml,
    max_epochs=max_epochs_ml, 
    device=device
    )
print(f'Final test loss = {test_loss_ml}\nFinal test MRR = {test_mrr_ml}')

Movie-lens file already exists
Movielens ratings shape: (1000209, 4)
Number of users: 6040
Number of items: 3706
Training Results - Epoch[1] Avg loss: 4.82
Validation Results - Epoch[1] Avg loss: 4.73
Validation Results - Epoch[1] MRR: 0.73
Final test loss = 4.733164878555248
Final test MRR = 0.7282241014799153


## Results Analysis

# Books

In [11]:
max_epochs_bk = 1
mrr_threshold_bk = 8

## HyperParameter Tuning

In [12]:
n_trials = 1

study_bk, df_tuning_results_bk = tune_params(
    model_name='MF', 
    dataset_name='books', 
    max_epochs=max_epochs_bk,
    n_trials=n_trials, 
    device=device,
    mrr_threshold=mrr_threshold_bk
    )

[I 2022-02-08 21:29:06,367] A new study created in memory with name: no-name-27a688b7-73df-41e1-abca-dbb60df4840c


Books file already exists
Books ratings shape: (433671, 3)
Number of users: 77805
Number of items: 185973
Training Results - Epoch[1] Avg loss: 23.44
Validation Results - Epoch[1] Avg loss: 23.81


[I 2022-02-08 21:38:28,331] Trial 0 finished with value: 23.80758975309719 and parameters: {'learning_rate': 0.1, 'optimizer': 'SGD', 'latent_dim': 500, 'batch_size': 512}. Best is trial 0 with value: 23.80758975309719.


Validation Results - Epoch[1] MRR: 0.61


In [13]:
best_params_bk = study_bk.best_params
df_tuning_results_bk.sort_values(by='value').head(15)

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_latent_dim,params_learning_rate,params_optimizer,state
0,0,23.80759,2022-02-08 21:29:06.368181,2022-02-08 21:38:28.331136,0 days 00:09:21.962955,512,500,0.1,SGD,COMPLETE


## Full Training
- Use the best hyperparameters to build the final model
- Train the final model on the train+validation data sets (full_train)
- Test it against the test set for final results

In [ ]:
test_loss_bk, final_model_bk, test_mrr_bk = final_train(
    model_name='MF', 
    dataset_name='books', 
    best_params=best_params_bk,
    max_epochs=max_epochs_bk, 
    device=device,
    mrr_threshold=mrr_threshold_bk
    )
print(f'Final test loss = {test_loss_bk}\nFinal test MRR = {test_mrr_bk}')

## Results Analysis

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=.